<a href="https://colab.research.google.com/github/anaptoro/git_geotec2/blob/main/Aula6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture 
# O comando capture é utilizado para que os outputs da célula não sejam exibidos, se quiser exibir, é só comentar a linha
!pip install geemap # pip é o comando de instalação no python, quando vc coloca a ! na frente do comando, vc está executando como se fosse no prompt de comando

In [2]:
%%capture
!pip install geopandas

In [3]:
# import geemap #aqui vc está importando a biblioteca
import ee

In [4]:
import geemap
import geopandas as gpd

In [5]:
Map = geemap.Map() #esse comando vai exigir uma autenticação

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=86ET5LgR-rdh3M7X07ox4AAe4_1OL69_QCAVuj3eRgo&tc=rpMIamwlT1m0pVFxa22vR9OywanIOhdi9-D4OiOjGyc&cc=1qX_Rb1W_5Iwe6xd8AzK8bDNK77UvmLPtx0YQdXlTC4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJqjLYZV5yr922TYUxIkUyidWMDiZVQFgNelAUIFTzfnh52-NiN12Ps

Successfully saved authorization token.


In [6]:
AE = ee.FeatureCollection('users/gleyce/ROI_Campina')

In [8]:
#carregando as duas classificações
RF1 = ee.Image('users/gleyce/RF1')
RF2 = ee.Image('users/gleyce/RF2')

In [22]:
lista_new=[10,20,30]
lista_raw=[1,2,3]
# //Reclassificando novamente para retornar classes para 0 até o número da última classe (4)
M2004 = RF1.remap(lista_raw,lista_new,0,'b1')# 3 classes

M2016 = RF2.remap(lista_raw,lista_new,0,'b1')#// 3 classes


In [23]:
igbpPalette = [
  'aec3d4', # nativa
  '152106', '225129', '369b47', '30eb5b', '387242', # forest
  '6a2325', 'c3aa69', 'b76031', 'd9903d', '91af40',  # pasto
  # '111149', # wetlands
  # 'cdb33b', # croplands
  # 'cc0013', # urban
  # '33280d', # crop mosaic
  # 'd7cdcc', # snow and ice
  # 'f7e084', # barren
  # '6f6f6f'  # tundra
]

Map = geemap.Map()
Map.addLayer(M2004,{'palette': igbpPalette, 'min': 10, 'max': 30},'M2004')
Map.addLayer(M2016,{'palette': igbpPalette, 'min': 10, 'max': 30},'M2016')
Map.centerObject(AE, 13)
Map

Map(center=[-21.619223403207844, -51.886861533829226], controls=(WidgetControl(options=['position', 'transpare…

In [24]:
#Para realizar a diferença entre os mapas precisamos reclassificar os valores de um dos mapas
RM2016 = M2016.remap([10,20,30],[100,200,300], 0, 'remapped');    
Map.addLayer(M2004,{'palette': igbpPalette, 'min': 10, 'max': 30},'M2004')
Map.addLayer(RM2016, { 'palette': igbpPalette, 'min': 100, 'max': 300,}, 'RM2016')
Map

Map(bottom=1177917.0, center=[-21.619223403207844, -51.886861533829226], controls=(WidgetControl(options=['pos…

In [25]:
# //Listar valores das diferentes classes para L5 e L8;
# //Listar valores das diferentes classes para L5 e L8;
classe0_L5 = 10 #Nativa
classe0_L8 = 100 #Nativa
classe1_L5 = 20 #umido
classe1_L8 = 200 #umido
classe2_L5 = 30 #ag
classe2_L8 = 300 #ag
# classe3_L5 = 3 #anual
# classe3_L8 = 400 #anual
# classe4_L5 = 4 #pastagem
# classe4_L8 = 500 #pastagem

In [26]:
#Achando o valor da diferença das classes no mapa
dif_class = classe0_L8-classe1_L5 #;//o que é agricultura anual em 2016 e era pastagem em 2004
print("Pastagem --> Agricultura Anual:", dif_class);
same_class = classe0_L8-classe0_L5 #;//o que permaneceu como agricultura anual entre 2004 e 2016
print("Agricultura Anual --> Agricultura Anual:", same_class);

Pastagem --> Agricultura Anual: 80
Agricultura Anual --> Agricultura Anual: 90


In [27]:
#Calculando a diferença entre os mapas do ano inicial e do ano final
diff = RM2016.subtract(M2004)

In [28]:
#Criação de raster de mudança de uso da terra de acordo com a classe de interesse

# Change = Change.updateMask(Change.eq(99))
Change = diff
Change = Change.updateMask(diff.eq(80)) #;//Nesse caso, o valor 396 é referente a mudança Pastagem --> Anual. Colocar o valor da sua classe de 

In [29]:
Map = geemap.Map()
Map.addLayer(Change, {},'Mudança')
Map.centerObject(AE, 13)
Map

Map(center=[-21.619223403207844, -51.886861533829226], controls=(WidgetControl(options=['position', 'transpare…

In [30]:
#Criação de raster de não mudança de uso da terra de acordo com a classe de interesse
nchange = diff.updateMask(diff.eq(90))#; //Nesse caso, o valor 90 significa que em 2006 era Anual e em 2016 também era Anual
Map.addLayer(nchange,{'palette':['green']},'Não Mudança')
Map

Map(bottom=1177917.0, center=[-21.619223403207844, -51.886861533829226], controls=(WidgetControl(options=['pos…